In [1]:
!pip install assemblyai

import assemblyai as aai
import os

In [2]:
aai.settings.api_key = "01754830854649d0915488dc8964f835"

In [3]:
config = aai.TranscriptionConfig(
    speaker_labels=True,
    sentiment_analysis=True,
    speech_model=aai.SpeechModel.best
)

In [4]:
local_path = local_path = local_path = os.path.expanduser("~/Desktop/EY_Project/Customer Service Sample Call - Product Refund [TubeRipper.com].mp3")


transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe(local_path)

if transcript.status == "error":
    raise RuntimeError(f"Transcription failed: {transcript.error}")

for utt in transcript.utterances:
    print(f"[Speaker {utt.speaker}] {utt.text}")

[Speaker A] Thank you for calling Coats and Gowns. My name is Sam, how can I help you?
[Speaker B] Oh yes, I bought a coat from you guys but I need to return it because it was the wrong size and my size is not available. I haven't received any email from you and I wonder if if my payment has been refunded. Seems like it does not reflect on my account yet.
[Speaker A] I do apologize for the inconvenience, but let me go ahead and check if there are some notes in the order details. May I ask, May I ask for the original online order number?
[Speaker B] Okay, let me see. All right. Please wait for a moment. Okay, hold on. Okay, I believe it's 017-14-245793.
[Speaker A] Thank you. Just to verify that 017-14-25793, correct?
[Speaker B] Yes, that's right.
[Speaker A] I do apologize, but the one that you provided is actually not accepting towards our database. Do you actually have a number that starts with triple zero, seven or seven? Or can ask for your first and last name so I can manually ch

In [5]:
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

results = []
for i, utt in enumerate(transcript.utterances):
    line = utt.text.strip()
    speaker = utt.speaker
    scores = analyzer.polarity_scores(line)
    sentiment = (
        "Positive" if scores['compound'] > 0.05 else
        "Negative" if scores['compound'] < -0.05 else
        "Neutral"
    )
    results.append({
        "Line": i + 1,
        "Speaker": speaker,
        "Text": line,
        "Sentiment": sentiment,
        "Compound Score": scores['compound']
    })

df = pd.DataFrame(results)

overall_sentiment = df['Sentiment'].value_counts().idxmax()

summary_row = {
    "Line": "Overall",
    "Speaker": "",
    "Text": "Conversation Summary",
    "Sentiment": overall_sentiment,
    "Compound Score": df["Compound Score"].mean()  
}
df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

df.to_excel("transcript3_sentiment.xlsx", index=False)

df.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aramishra/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Line,Speaker,Text,Sentiment,Compound Score
0,1,A,Thank you for calling Coats and Gowns. My name...,Positive,0.6369
1,2,B,"Oh yes, I bought a coat from you guys but I ne...",Neutral,-0.0129
2,3,A,"I do apologize for the inconvenience, but let ...",Positive,0.4310
3,4,B,"Okay, let me see. All right. Please wait for a...",Positive,0.7184
4,5,A,"Thank you. Just to verify that 017-14-25793, c...",Positive,0.3612
